In [1]:
# %matplotlib inline
from __future__ import annotations
import numpy as np
import typing
from utils import (get_session, get_stim, get_fs, get_raw_responses)

import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import gridspec
from scipy import stats, signal
from scipy.signal import hilbert
from scipy.signal import resample
from numpy.lib.recfunctions import append_fields, merge_arrays
from nilearn import plotting
from scipy.stats import circmean
import pycircstat
from matplotlib.backends.backend_pdf import PdfPages
import dask

import sys
from SubjectLevel.par_funcs import *
import par_funcs
sys.path.append('/home1/anup.das')
import pickle
import glob
import mat73

from tarjan import tarjan
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import LinearRegression as LR
import matplotlib.colors as clrs
import matplotlib.cm as cmx
import nilearn.plotting as ni_plot
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pylab as pyl
from PIL import Image
import os
from scipy.spatial.distance import pdist, squareform
from SubjectLevel.par_funcs_fine import *
from scipy.io import loadmat

from ptsa.data.readers import EEGReader
from ptsa.data.filters import ResampleFilter
import warnings; warnings.simplefilter('ignore')

from ptsa.data.filters import ButterworthFilter
from ptsa.data.filters import MorletWaveletFilter
from ptsa.data.filters import ResampleFilter
from ptsa.data.timeseries import TimeSeries

import mne
from mne.viz import plot_alignment, snapshot_brain_montage
from mne import create_info, EpochsArray
from mne.baseline import rescale
from mne.time_frequency import (tfr_multitaper, tfr_stockwell, tfr_morlet,
                                tfr_array_morlet)
from mne.viz import centers_to_edges

from joblib import Parallel, delayed
import multiprocessing
import cluster_helper.cluster
import time
from collections import defaultdict
from types import SimpleNamespace
from collections import OrderedDict

/home1/anup.das/.conda/envs/py3_env/lib/python3.7/site-packages/ptsa/data/readers/index.py:39: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)


In [3]:
session_id = '39491886'
session = get_session(session_id)
session

onsets, offsets, stim_params = get_stim(session)

num_trials, num_dots, _ = stim_params['dot_locations'].shape
print('trial onsets in ms: {}'.format(onsets.shape))
print('trial offsets in ms: {}'.format(offsets.shape))
print('{} trials in the session, average duration {:.1f} ms'.format(num_trials, np.mean(offsets-onsets)))
print('{} dots presented in each trial'.format(num_dots))

trial onsets in ms: (401,)
trial offsets in ms: (401,)
401 trials in the session, average duration 2002.1 ms
40 dots presented in each trial


In [5]:
#create coordinates of channels
xg = np.empty((32, 32))
yg = np.empty((32, 32))

hor_pitch = 26.5
ver_pitch = 29

for i_coord in range(0, 32, 1):
    for j_coord in range(0, 32, 1):
        
        xg[i_coord, j_coord] = -i_coord*ver_pitch
        yg[i_coord, j_coord] = j_coord*hor_pitch

xg_reshaped = np.reshape(xg, (1024, 1))
yg_reshaped = np.reshape(yg, (1024, 1))
coords = np.array(np.hstack((xg_reshaped, yg_reshaped)))

In [6]:
fs = get_fs(session)

diffvals = []
for i_trial in range(0, len(offsets), 1):
    diffvals.append(offsets[i_trial] - onsets[i_trial])

eeg_all_chans = []

for i_trial in range(0, len(onsets), 1):
    tic, toc = onsets[i_trial]-1000, onsets[i_trial] + np.ceil(np.max(diffvals)) +1000 # a time interval covering the first trial
    if (toc-tic > 4082.5):
        toc -= 1
    responses = get_raw_responses(session, tic, toc)
    
    time_axis_length = np.squeeze(responses[0, :].shape)
    new_length = int(np.round(time_axis_length * 500 / float(fs)))
    responses_resampled = resample(responses.T, new_length).T
    eeg_all_chans.append(responses_resampled)

In [8]:
#create spatial channels
channel_idxs = np.arange(1024)

eeg_all_chans = np.array(eeg_all_chans)

sweep_offsets = [(0, 0), (1, 0), (1, 1), (0, 1)]
idx_map = np.empty((32, 32), dtype=int)
for i, (y, x) in enumerate(sweep_offsets):
    idx_map[y::2, x::2] = channel_idxs[i*256:(i+1)*256].reshape(16, 16)

idx_map_reshaped = idx_map.reshape(np.shape(eeg_all_chans)[1],1)
eeg_all_chans_spatial = []
for i_trial in range(0, np.shape(eeg_all_chans)[0], 1):
    data_mat_tmp = np.empty((np.shape(eeg_all_chans)[1], np.shape(eeg_all_chans)[2]))
    data_mat_orig = eeg_all_chans[i_trial, :, :]
    for i_elec in range(0, np.shape(eeg_all_chans)[1], 1):
        data_mat_tmp[i_elec, :] = data_mat_orig[idx_map_reshaped[i_elec], :]
    eeg_all_chans_spatial.append(data_mat_tmp)
eeg_all_chans_spatial = np.array(eeg_all_chans_spatial)
with open('/home1/anup.das/dense-session-share_081224/Results/resampled_eeg_spatial_39491886.pkl', 'wb') as f:
        pickle.dump(eeg_all_chans_spatial, f, protocol=4)

In [5]:
with open('/home1/anup.das/dense-session-share_081224/Results/resampled_eeg_spatial_39491886.pkl', 'rb') as f:
    eeg_all_chans_spatial = np.array(pickle.load(f))

In [9]:
#implement RAM dask based traveling waves analysis
import cmldask
from dask.distributed import wait, progress
from cmldask import CMLDask as da

client = da.new_dask_client_slurm('traveling_waves', '8GB', max_n_jobs = 50, walltime = '100000', log_directory = '/home1/anup.das/logs')

Unique port for anup.das is 51437
{'dashboard_address': ':51437'}
To view the dashboard, run: 
`ssh -fN anup.das@rhino2.psych.upenn.edu -L 8000:192.168.86.144:51437` in your local computer's terminal (NOT rhino) 
and then navigate to localhost:8000 in your browser


In [14]:
#implement RAM dask based traveling waves analysis
from scipy.signal import butter, lfilter

session_id = '39491886'
session = get_session(session_id)

fs = get_fs(session)

onsets, offsets, stim_params = get_stim(session)

num_trials, num_dots, _ = stim_params['dot_locations'].shape

dot_locations = stim_params['dot_locations']
dot_colors = stim_params['dot_colors']
fs = get_fs(session)

locations_all = np.reshape(stim_params['dot_locations'], (401*40, 2))
locations_unique = np.unique(locations_all, axis=0)

DOT_TIME = 50
TIME_PAST_STIM = [40,90]
TRIAL_TIME_PAD = [0.1,0.1+TIME_PAST_STIM[1]-DOT_TIME]

dot_starts = []
dot_ends = []

for trial in range(dot_locations.shape[0]):

    dot_st_frame = []
    dot_end_frame = []
    
    # for each dot
    for d in range(dot_locations.shape[1]):

        # get start and end frame
        st_temp = np.int16(np.ceil((d*DOT_TIME + TRIAL_TIME_PAD[0] + TIME_PAST_STIM[0]) * (fs/1000)))
        end_tmp = np.int16(st_temp + np.ceil(TIME_PAST_STIM[1] * (fs/1000)))
        
        dot_st_frame.append(st_temp)
        dot_end_frame.append(end_tmp)
        
    dot_starts.append(dot_st_frame)
    dot_ends.append(dot_end_frame)
    
dot_starts = np.array(dot_starts).astype(float)
dot_ends = np.array(dot_ends).astype(float)


channel_idxs = np.arange(1024)

sweep_offsets = [(0, 0), (0, 1), (1, 0), (1, 1)]
idx_map = np.empty((32, 32), dtype=int)
for i, (y, x) in enumerate(sweep_offsets):
    idx_map[y::2, x::2] = channel_idxs[i*256:(i+1)*256].reshape(16, 16)

idx_map_reshaped = idx_map.reshape(len(channel_idxs),1)


#filter signals and calculate power
def butter_bandpass(lowcut, highcut, fs, order=3):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y








#run traveling waves analysis
#reshape the data matrix into channel by trial by frame dimensional matrix
eeg = eeg_all_chans_spatial.transpose(1, 0, 2)

res_global = {}
task_phase = 'STIMULI'

print('Band pass EEG')
time_frame_start=500
time_frame_end=np.shape(eeg_all_chans_spatial)[2] - 500
thetas = np.radians(np.arange(0, 360, 5))
rs = np.radians(np.arange(0, 180/29, 0.5))
theta_r = np.stack([(x, y) for x in thetas for y in rs])
params = np.stack([theta_r[:, 1] * np.cos(theta_r[:, 0]), theta_r[:, 1] * np.sin(theta_r[:, 0])], -1)
f=par_funcs.circ_lin_regress

#allowed electrodes
allowed_elecs = np.ones((len(coords)), dtype=bool)
saturating_ch_inds = np.hstack([range(0,8), range(1016,1024)])
n_allowed_elecs_inds = np.where(idx_map_reshaped == saturating_ch_inds)[0]
allowed_elecs[n_allowed_elecs_inds] = False

i=1
print('Traveling wave analysis')
#while 'cluster{}'.format(i) in self.res['clusters'].columns:
while 'cluster{}'.format(i) in 'cluster1':
    res={}
    clusterphase=[]
    clusterpower=[]
    clustereeg=[]
    cluster='cluster{}'.format(i)
    for j in range(len(allowed_elecs)):
        if allowed_elecs[j]:
            feeg=butter_bandpass_filter(eeg[j], 30 *.85, 90/.85, 500, order=3)
            clusterphase.append(np.angle(hilbert(feeg, N=feeg.shape[-1], axis=-1)))
            clusterpower.append(np.abs(hilbert(feeg, N=feeg.shape[-1], axis=-1)))
            clustereeg.append(feeg)
    res['power']=np.stack(clusterpower)[:,:,time_frame_start:time_frame_end].astype('float32') 
    res['phase']=np.stack(clusterphase)[:,:,time_frame_start:time_frame_end].astype('float32') 
    res['eeg']=np.stack(clustereeg)[:,:,time_frame_start:time_frame_end].astype('float32') 
    
    xyz = coords
    xyz = xyz[allowed_elecs]
    elec_dists = squareform(pdist(xyz))
    near_adj_matr = (elec_dists < 80)

    local_angle={}
    local_sf={}
    local_rs={}
    local_off={}
    for j in tqdm(range(0,len(xyz))):
        if sum(near_adj_matr[j])>3:
            norm_coords = xyz[near_adj_matr[j],:]

            num_iters = int(res['phase'][near_adj_matr[j]].T.shape[0])
            
            data_as_list = zip(res['phase'][near_adj_matr[j]].T,np.array([norm_coords]*num_iters), [theta_r]*num_iters, [params]*num_iters)
            arg1 = [(x[0]) for x in data_as_list]

            data_as_list = zip(res['phase'][near_adj_matr[j]].T,np.array([norm_coords]*num_iters), [theta_r]*num_iters, [params]*num_iters)
            arg2 = [(x[1]) for x in data_as_list]

            data_as_list = zip(res['phase'][near_adj_matr[j]].T,np.array([norm_coords]*num_iters), [theta_r]*num_iters, [params]*num_iters)
            arg3 = [(x[2]) for x in data_as_list]

            data_as_list = zip(res['phase'][near_adj_matr[j]].T,np.array([norm_coords]*num_iters), [theta_r]*num_iters, [params]*num_iters)
            arg4 = [(x[3]) for x in data_as_list]

            futures = client.map(f, arg1, arg2, arg3, arg4)
            progress(futures)
            res_as_list = client.gather(futures)
    
            local_angle[j]=np.stack([x[0] for x in res_as_list], axis=0).astype('float32') 
            local_sf[j]=np.stack([x[1] for x in res_as_list], axis=0).astype('float32') 
            local_rs[j]=np.stack([x[2] for x in res_as_list], axis=0).astype('float32')
            local_off[j]=np.stack([x[3] for x in res_as_list], axis=0).astype('float32')  
    
    res['direction'] = local_angle 
    res['spatial_freuency']= local_sf
    res['rs']= local_rs
    res['offs']= local_off
    if cluster not in res_global:
        res_global[cluster]={}
    res_global[cluster][task_phase]=res
    i+=1
    
with open('/home1/anup.das/dense-session-share_081224/Results/res_global_gamma_39491886.pkl', 'wb') as f:
        pickle.dump(res_global, f)

Band pass EEG
Traveling wave analysis


100%|██████████| 1008/1008 [1:46:22<00:00,  6.33s/it] 
